In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,23330
2,Huelva,Confirmados PDIA 14 días,4012
3,Huelva,Tasa PDIA 14 días,"781,8071983942942"
4,Huelva,Confirmados PDIA 7 días,1417
5,Huelva,Total Confirmados,23560
6,Huelva,Curados,12325
7,Huelva,Fallecidos,219


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  23330.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6176.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 448 personas en los últimos 7 días 

Un positivo PCR cada 150 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,23330.0,1417.0,4012.0,513170.0,276.126820,781.807198,59.0
Huelva-Costa,14169.0,845.0,2399.0,289548.0,291.834169,828.532748,39.0
Huelva (capital),6176.0,321.0,958.0,143837.0,223.169282,666.031689,22.0
Condado-Campiña,6458.0,441.0,1227.0,156231.0,282.274325,785.375502,15.0
Lepe,1879.0,115.0,330.0,27880.0,412.482066,1183.644189,7.0
Sierra de Huelva-Andévalo Central,2288.0,70.0,239.0,67391.0,103.871437,354.646763,4.0
Rociana del Condado,287.0,30.0,90.0,7939.0,377.881345,1133.644036,3.0
Bonares,171.0,14.0,53.0,6060.0,231.023102,874.587459,3.0
Isla Cristina,1919.0,118.0,297.0,21393.0,551.582293,1388.304586,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Berrocal,19.0,4.0,7.0,303.0,1320.132013,2310.231023,0.0
Villablanca,155.0,33.0,66.0,2885.0,1143.847487,2287.694974,2.0
Villalba del Alcor,436.0,25.0,64.0,3366.0,742.721331,1901.366607,0.0
Paymogo,71.0,10.0,20.0,1162.0,860.585198,1721.170396,1.0
Cabezas Rubias,28.0,2.0,10.0,706.0,283.286119,1416.430595,0.0
Isla Cristina,1919.0,118.0,297.0,21393.0,551.582293,1388.304586,3.0
San Bartolomé de la Torre,187.0,12.0,49.0,3761.0,319.064079,1302.844988,0.0
Ayamonte,1136.0,98.0,274.0,21104.0,464.366945,1298.332070,1.0
Palma del Condado (La),951.0,48.0,139.0,10801.0,444.403296,1286.917878,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cerro de Andévalo (El),131.0,1.0,15.0,2327.0,42.973786,644.606790,0.0,0.066667
Villarrasa,127.0,1.0,10.0,2211.0,45.228403,452.284034,0.0,0.100000
Jabugo,64.0,1.0,9.0,2260.0,44.247788,398.230088,0.0,0.111111
Encinasola,116.0,1.0,7.0,1310.0,76.335878,534.351145,0.0,0.142857
Alosno,118.0,3.0,17.0,3933.0,76.277651,432.240020,0.0,0.176471
Nerva,145.0,3.0,16.0,5169.0,58.038305,309.537628,0.0,0.187500
Zalamea la Real,82.0,3.0,15.0,3054.0,98.231827,491.159136,0.0,0.200000
Cabezas Rubias,28.0,2.0,10.0,706.0,283.286119,1416.430595,0.0,0.200000
Paterna del Campo,110.0,7.0,29.0,3457.0,202.487706,838.877640,0.0,0.241379
